In [1]:
!pip install -q langchain numexpr sentencepiece plotly diffusers


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from langchain.agents import load_tools
from langchain.chat_models import ChatHuggingFace, ChatOpenAI
from langchain_core.tools import Tool
import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv(override=True)

from scripts.modified_calculator import LLMMathChain
from scripts.run_agents import answer_questions

In [4]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"

In [5]:
from scripts.evaluation import load_benchmark
import datasets

eval_df = load_benchmark()
eval_df["true_answer"] = eval_df["true_answer"].astype(str)
eval_ds = datasets.Dataset.from_pandas(eval_df)

OUTPUT_DIR = "output"

In [6]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    max_new_tokens=3000,
)
llm_engine = ChatHuggingFace(llm=llm)


def call_llm(input: str, stop=["Observation", "Final Answer"]) -> str:
    return llm_engine.invoke(input, stop=stop).content


call_llm("I should multiply 2 by 3.6452.")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


' To multiply 2 by the number 3.6452, you can simply multiply each digit of 2 by each digit of 3.6452 and sum up the results. However, since 2 is a single digit number, you can just multiply it by each digit of 3.6452 and sum up the results:\n\n2 \\* 3 = 6\n2 \\* 6 = 12 (remember to carry the "1" to the next digit)\n2 \\* 4 = 8 (add the carried "1" to this digit, making it 9)\n2 \\* 5 = 10 (add the carried "1" to this digit, making it 11, but since we only have one digit after the decimal point, we need to round it to the nearest whole number, so the final result is 7.6452, but since we only want to keep 4 decimal places, we round it to 7.6452)\n\nTherefore, 2 \\* 3.6452 = 7.6452.'

### Define tools

In [7]:
from transformers.tools.base import CalculatorTool, Tool

from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
langchain_serpapi = SerpAPIWrapper(params=params)


class SearchTool(Tool):
    name = "search"
    description = "Search the web for information."

    inputs = {"query": str}
    outputs = [str]

    def __call__(self, query: str) -> str:
        return langchain_serpapi.run(query)

In [8]:
search = SearchTool()

In [9]:
TOOLBOX = [CalculatorTool(), SearchTool()]

# Define agents

In [10]:
from transformers import CodeAgent, ReactAgent

code_agent = CodeAgent(llm_callable=call_llm, toolbox=TOOLBOX)
react_agent = ReactAgent(llm_callable=call_llm, toolbox=TOOLBOX)

agents = {
    "code": code_agent,
    "react": react_agent,
}


No function template is defined for this tokenizer - using a default function template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.tool_description_template` to an appropriate template. 


In [11]:
async def call_transformers(agent, question: str) -> str:
    result = agent.run(question)
    return {
        "output": str(result),
        "intermediate_steps": agent.messages.copy(),
    }


# await answer_questions(
#     eval_ds,
#     code_agent,
#     "code_agent",
#     output_folder=OUTPUT_DIR,
#     agent_call_function=call_transformers,
#     key_for_answer="true_answer",
# )

In [12]:
from scripts.run_agents import run_full_tests

await run_full_tests(
    eval_ds,
    agents,
    output_folder=OUTPUT_DIR,
    agent_call_function=call_transformers,
    key_for_answer="true_answer",
)

100%|██████████| 90/90 [00:00<00:00, 13313.84it/s]


,agent_name,question,prediction,intermediate_steps,parsing_error,iteration_limit_exceeded,agent_error,start_time,end_time,gt_answer,task
0,code,Mimi picked up 2 dozen seashells on the beach....,16.0,[],False,False,None,2024-03-05 15:45:14,2024-03-05 15:45:22,16.0,gsm8k
1,code,Frankie's parents let him have many pets. He h...,None,[],False,False,None,2024-03-05 15:45:22,2024-03-05 15:45:34,19.0,gsm8k
2,code,"Olaf collects colorful toy cars. At first, his...",None,[],False,False,None,2024-03-05 15:45:34,2024-03-05 15:45:41,196.0,gsm8k
3,code,Emma's bank account has $100 in it. Each day o...,4,[],False,False,None,2024-03-05 15:45:41,2024-03-05 15:45:48,4.0,gsm8k
4,code,"Ezekiel hikes as a hobby. This past summer, he...",25.0,[],False,False,None,2024-03-05 15:45:48,2024-03-05 15:45:53,15.0,gsm8k
...,...,...,...,...,...,...,...,...,...,...,...
175,react,How many at bats did the Yankee with the most ...,None,"[{'role': 'user', 'content': 'Solve the follow...",False,False,None,2024-03-05 18:11:53,2024-03-05 18:12:17,519,GAIA
176,react,In Audre Lorde’s poem “Father Son and Holy Gho...,None,"[{'role': 'user', 'content': 'Solve the follow...",False,False,None,2024-03-05 18:12:17,2024-03-05 18:12:44,2,GAIA
177,react,Where were the Vietnamese specimens described ...,St. Petersburg,"[{'role': 'user', 'content': 'Solve the follow...",False,False,None,2024-03-05 18:12:44,2024-03-05 18:13:03,Saint Petersburg,GAIA
178,react,What country had the least number of athletes ...,None,"[{'role': 'user', 'content': 'Solve the follow...",False,False,None,2024-03-05 18:13:03,2024-03-05 18:13:38,CUB,GAIA


# Evaluate

In [23]:
import glob

answer_file_path = f"{OUTPUT_DIR}/answers.jsonl"

result_df = pd.concat([pd.read_json(f) for f in glob.glob(f"{OUTPUT_DIR}/*.json")])
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

### Exact match for GSM8K

In [24]:
from scripts.evaluation import score_any_match

results_math = result_df.loc[result_df["task"] == "gsm8k"].copy()
results_math["exact_match"] = -1
results_math["exact_match"] = results_math.apply(
    lambda row: score_any_match(row["prediction"], float(row["gt_answer"])), axis=1
)

In [25]:
results_math["agent_name"].value_counts()

agent_name
code          40
react         40
react_prev    40
Name: count, dtype: int64

In [26]:
results_math.groupby(["agent_name", "task"])["exact_match"].mean()

agent_name  task 
code        gsm8k    0.675
react       gsm8k    0.375
react_prev  gsm8k    0.425
Name: exact_match, dtype: float64

### LLM as a judge for others

In [27]:
answers = result_df.loc[result_df["task"] != "gsm8k"].to_dict(orient="records")

In [28]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [29]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = f"{OUTPUT_DIR}/evaluation.jsonl"

run_evaluation = True
if run_evaluation:
    results = await evaluate_answers(
        answers,
        eval_chat_model,
        "GPT4",
        EVALUATION_PROMPT_TEMPLATE,
        output_file_path=output_file_path,
    )

    print("Evaluation is complete!")

Previous evaluations:
Launching evaluation for 0 examples...


0it [00:00, ?it/s]

Evaluation is complete!


In [30]:
results_nonmath = pd.DataFrame.from_dict(results)


def interpret_score(eval_score):
    try:
        return (float(eval_score) - 1) / 4
    except:
        return 0


results_nonmath["eval_score_GPT4"] = results_nonmath["eval_score_GPT4"].apply(
    interpret_score
)
results_nonmath.groupby("agent_name")["eval_score_GPT4"].mean()

agent_name
code          0.255
react         0.345
react_prev    0.335
Name: eval_score_GPT4, dtype: float64

### Aggregate both

In [31]:
result_df = pd.concat([results_math, results_nonmath])
result_df["aggregate_score"] = result_df["exact_match"].fillna(0) + result_df[
    "eval_score_GPT4"
].fillna(0)
result_df.groupby(["agent_name", "task"])["aggregate_score"].mean()

agent_name  task    
code        GAIA           0.125
            HotpotQA    0.341667
            gsm8k          0.675
react       GAIA          0.1375
            HotpotQA    0.483333
            gsm8k          0.375
react_prev  GAIA          0.0875
            HotpotQA         0.5
            gsm8k          0.425
Name: aggregate_score, dtype: object

### Display

In [36]:
import plotly.express as px

aggregate = (
    result_df.groupby(["agent_name", "task"])[["aggregate_score"]].mean().reset_index()
)
fig = px.bar(aggregate, x="agent_name", y="aggregate_score", color="task")
fig.update_layout(
    barmode="group", width=len(aggregate["agent_name"].unique()) * 200 + 200
)

/Users/aymeric/venvs/ml/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

